<a href="https://colab.research.google.com/github/yangsu0/class_machine_learning/blob/master/ML_Assignment10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np

file_data   = "mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images

#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) -min(data))

    return(data_normalized)

#
# example of distance function between two vectors x and y
#
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)

#
# make a matrix each column of which represents an images in a vector form
#
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:

    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)

    list_label[count]       = label
    list_image[:, count]    = im_vector

    count += 1





In [0]:
#prepare one-hot-encoded label list
target = list_label
num = np.unique(target)

#one_hot_label = np.eye(10)[num]
num = num.shape[0]
encoded_label = np.eye(num)[target]
encoded_label = encoded_label.T

In [0]:
#prepare train & test set
train_image = np.empty((size_row * size_col, 6000), dtype=float)
test_image = np.empty((size_row * size_col, 4000), dtype=float)
test_label = np.empty((10, 4000), dtype=float)
train_label = np.empty((10, 6000), dtype=float)

for i in range(num_image):
  if i>=6000:
    test_image[:, i-6000] = normalize(list_image[:, i])
    test_label[:, i-6000] = encoded_label[:, i]
  else:
    train_image[:, i] = normalize(list_image[:, i])
    train_label[:, i] = encoded_label[:, i]

In [0]:
#weight & bias parameters
u_list = []
v_list = []
w_list = []

u=np.random.randn(196,784)
v=np.random.randn(49,196)
w=np.random.randn(10,49)
b1 = np.zeros((196, 1))
b2 = np.zeros((49, 1))
b3 = np.zeros((10, 1))

lamb = 0.75

In [0]:
#define functions
def sigmoid(z):
  return 1/(1+np.exp(-z))

def sigmoid_prime(z):
  return sigmoid(z) * (1 - sigmoid(z))

def object_func(label, sig, m):
  return -np.mean(label*np.log(sig) + (1-label)*np.log(1-sig)) + (lamb/(2*m))*(np.sum(np.square(u)) + np.sum(np.square(v)) + np.sum(np.square(w)))

In [0]:
loss_train_list = []
loss_test_list = []
acc_train_list = []
acc_test_list = []

In [0]:
#y' = u*x
#y = sigmoid(y')
#z' = v*y
#z = sigmoid(z')
#h' = w*z
#h = sigmoid(h')

#dh = (h-train_label)/h*(1-h)
#dh'= np.multiply(dh, sigmoid_prime(dh))

#####dw = np.dot(dh',z.T)

#dz = np.dot(w.T,dh')
#dz' = np.multiply(dz, sigmoid_prime(dz))

#####dv = np.dot(dz', y.T)

#dy = np.dot(v.T,dz')
#dy'= np.multiply(dy, sigmoid_prime(dy))

#####du = np.dot(dy',train_image.T)

def gradient(h_,z_,y_,h,z,y):
  dh = (h-train_label)/h*(1-h)
  dh_ = np.multiply(dh, sigmoid_prime(h_))
  dw = np.dot(dh_,z.T)+(lamb)*w
  db3 = np.sum(dh_, axis=1, keepdims=True)
  dz = np.dot(w.T,dh_)
  dz_ = np.multiply(dz, sigmoid_prime(z_))
  dv = np.dot(dz_, y.T)+(lamb)*v
  db2 = np.sum(dz_, axis=1, keepdims=True)
  dy = np.dot(v.T,dz_)
  dy_= np.multiply(dy, sigmoid_prime(y_))
  du = np.dot(dy_,train_image.T)+(lamb)*u
  db1 = np.sum(dy_, axis=1, keepdims=True)

  return (1/6000) * du, (1/6000) * dv,  (1/6000) * dw, (1/6000) * db1, (1/6000) * db2, (1/6000) * db3,

In [0]:
correct = [] #collect the index of correct images
missed = {} #collect the index of missed images and derived label 
def accuracy(label, sig):
  data_len = label.shape[1]
  max_index = sig.argmax(axis=0)
  cnt = 0

  for i in range(data_len):
    if label[max_index[i],i] == 1:
      cnt += 1
      if data_len == 4000: #test_image set
          correct.append(i)
    else:
      if data_len == 4000:
        missed[i] = max_index[i] #{#1(data):#2, #2:#4,..}
  return (cnt/data_len) * 100


In [0]:
step_size = 0.285

#Loop while the object function convergence 
past_loss = 0
while(1):
  y_ = np.dot(u, train_image) + b1
  z_ = np.dot(v, sigmoid(y_)) + b2
  h_ = np.dot(w, sigmoid(z_)) + b3
  
  loss = object_func(train_label, sigmoid(h_), train_label.shape[1])
  loss_train_list.append(loss)
  acc = accuracy(train_label, sigmoid(h_))
  acc_train_list.append(acc)
  newu, newv, neww, nb1, nb2, nb3 = gradient(h_,z_,y_,sigmoid(h_), sigmoid(z_), sigmoid(y_))
  
  y_test = np.dot(u, test_image) + b1
  z_test = np.dot(v, sigmoid(y_test)) + b2
  h_test = np.dot(w, sigmoid(z_test)) + b3

  loss_test = object_func(test_label, sigmoid(h_test), test_label.shape[1])
  loss_test_list.append(loss_test)
  acc_test = accuracy(test_label, sigmoid(h_test))
  acc_test_list.append(acc_test)

  u -= step_size * newu
  v -= step_size * newv
  w -= step_size * neww
  b1 -= step_size * nb1
  b2 -= step_size * nb2
  b3 -= step_size * nb3
  
  #until convergence
  print(loss)
  if abs(loss-past_loss) < 0.000001:
    break
  past_loss = loss